# Exploring phonological areality using a naive Bayes classifier

This homework replicates the analyses of Michael et al. 2014, who use a naive Bayes classifier to probe areality in the circum-Andean region, based on phoneme inventories.  

The central dataset is the [South American Phonological Inventory Database (SAPhon)](http://linguistics.berkeley.edu/~saphon/en/).  The data have been downloaded for you, in [YAML](https://en.wikipedia.org/wiki/YAML) format.  There is YAML support in Python, which is used here, so our treatment of YAML is roughly analogous to our treatment of XML: someone else has gone to the trouble of creating reliable and flexible routines to read files in this format, so we'll use their routines to keep life easy.

We will take the same approach to the naive Bayes classifier itself - again, [the relevant routines have been written for you](https://scikit-learn.org/stable/modules/naive_bayes.html) and are available in [scikit-learn](https://scikit-learn.org/stable/index.html), a standard machine learning library for Python.  This allows us to focus our attention on the high-level questions of data, training, testing, and generalization, in the specific case of phonological areality in South America.

Thanks to [Geoff Bacon](https://geoffbacon.github.io/) for creating the first draft of this homework.

In [1]:
import glob
from io import StringIO
import re
import yaml
import pandas as pd
from sklearn.naive_bayes import BernoulliNB
import seaborn as sns
import matplotlib.pyplot as plt
import math
from scipy.stats import gaussian_kde
import numpy as np

### Read in SAPhon data

- Read in the YAML files of the SAPhon dataset.  Each file corresponds to a language, and the critical information we care about is: (a) the language's name, (b) its ISO code, and (c) the list of phonemes in the language.
- Then store this information in a Pandas dataframe.

In [2]:
# read in SAPhon data.
# we construct the list `features`, which is a list of dicts, each dict containing
# the relevant information from the files.
features = []
filenames = glob.glob("data/saphon/*.yaml")
for filename in filenames:
    with open(filename) as file:
        data = list(yaml.safe_load_all(file))[0]
    this_language = {}
    this_language["name"] = data["name"]
    this_language["iso"] = data["iso_codes"][0]
    phonemes = data["phonemes"]
    for phoneme in phonemes:
        this_language[phoneme] = 1
    features.append(this_language)

features

[{'name': 'Abipon',
  'iso': 'axb',
  'p': 1,
  't': 1,
  'c': 1,
  'k': 1,
  'q': 1,
  'm': 1,
  'n': 1,
  'ɲ': 1,
  'ɣ': 1,
  'ʁ': 1,
  'h': 1,
  'l': 1,
  'ɾ': 1,
  'i': 1,
  'iː': 1,
  'e': 1,
  'eː': 1,
  'o': 1,
  'oː': 1,
  'ə': 1,
  'əː': 1,
  'a': 1,
  'aː': 1},
 {'name': 'Achagua',
  'iso': 'aca',
  'p': 1,
  'b': 1,
  't': 1,
  'd': 1,
  'k': 1,
  'kʷ': 1,
  'ʔ': 1,
  'ts': 1,
  'tʃ': 1,
  'm': 1,
  'n': 1,
  's': 1,
  'ʃ': 1,
  'ʒ': 1,
  'h': 1,
  'j': 1,
  'w': 1,
  'l': 1,
  'ɾ': 1,
  'i': 1,
  'iː': 1,
  'u': 1,
  'uː': 1,
  'e': 1,
  'eː': 1,
  'o': 1,
  'oː': 1,
  'a': 1,
  'aː': 1},
 {'name': 'Aché',
  'iso': 'guq',
  'p': 1,
  'b': 1,
  't': 1,
  'd': 1,
  'k': 1,
  'ɡ': 1,
  'ʔ': 1,
  'tʃ': 1,
  'dʒ': 1,
  'mb': 1,
  'nd': 1,
  'ŋɡ': 1,
  'ndʒ': 1,
  'm': 1,
  'n': 1,
  'ɲ': 1,
  'ɸ': 1,
  'β': 1,
  'v': 1,
  'j': 1,
  'w': 1,
  'ɾ': 1,
  'i': 1,
  'ĩ': 1,
  'ɨ': 1,
  'ɨ̃': 1,
  'u': 1,
  'ũ': 1,
  'e': 1,
  'ẽ': 1,
  'o': 1,
  'õ': 1,
  'a': 1,
  'ã': 1},
 {'n

In [3]:
# now we turn the `features` list into a dataframe.
# YOUR CODE GOES HERE.
features = pd.DataFrame()  # REPLACE WITH YOUR CODE.
features.head()

""


In [4]:
# now clean the data up a little.
# first replace NaN by 0 - this denotes absence of a given phoneme.
# YOUR CODE GOES HERE.
features.head()

""


In [5]:
# now focus on the columns that hold phonemes, and convert the floats to ints.
# YOUR CODE GOES HERE.
features.head()

""


In [6]:
# there's a phoneme called "N", which will conflict with the name of one of the three classes in 
# michael et al.'s two-core analysis. 
# so to simplify matters, let's rename the phoneme to something else.
features.rename({"N": "NN"}, axis=1, inplace=True)
phoneme_columns[phoneme_columns.index("N")] = "NN"
features.head()

NameError: name 'phoneme_columns' is not defined

### Read in classification data

- Now read in the classification data, specifying which areal classes the various languages fall into.  
- The labeled data for this classification were copied from the publisher's webpage for the article.  Appendix C1 (data in file 'appendix_c1.txt') provides classes for the one-core analysis;  we will focus on this.  If you wish to extend to the two-core analysis, the relevant data are from Appendix C2, and can be found in the file 'appendix_c2.txt'.

In [ ]:
# read in classification data for the one-core analysis.
# use regular expressions to wrangle the txt file into a csv, then read with Pandas.
with open("data/appendix_c1.txt") as file:
    contents = file.read()
contents = re.sub(r" ", ",", contents)
contents = re.sub(r"\n\n", "\n", contents)
contents = re.sub(r"^([a-z])", r",\1", contents, flags=re.MULTILINE)
labels = pd.read_csv(StringIO(contents), names=["label", "iso", "NBC_score"])
labels.head()

In [ ]:
# we retain the NBC score because this is what we want to replicate.  we will check our results against it.
# now merge in the features data to create a single dataframe, with both features and labels 
# for the one core analysis.
# YOUR CODE GOES HERE.
one_core_data.head()

### One core analysis

In this homework we will replicate only the one-core analysis.

In the one-core analysis, there are two classes, or labels, for languages: A (Andean core) and C (control class, i.e. not Andean core).  Not all languages have a label - those languages that have a class label constitute our **training set**, and those that don't have a class label constitute our **test set**.  The essence of Michael et al.'s core-plus-periphery idea is that they train a classifier on a training set: a set of languages that we are fairly certain either do (label A) or don't (label C) belong to the Andean core.  They then ask which other languages, in the test set, are also rated as "core-like" by the classifier: that is, to which other languages the classifier assigns a high probability of belonging to the core (label A).

The input to the classifier is the phoneme inventory of a specific language, encoded as a binary vector (i.e. each element is either 0 or 1).  This format is already given to us in the dataframe above.  We denote this input feature vector by $x_1,...,x_n$, assuming there are $n$ features total (here, each feature is a phoneme that either is (1) or is not (0) in the inventory of the language).  Given input $x_1,...,x_n$, the classifier  determines the probability that the class $y$ is A for Andean core vs. C for control.  It does so using the conditional independence assumption that puts the "naive" in naive Bayes:

\begin{equation}
P(y | x_1,...,x_n) \propto P(y) \prod_{i=1}^n P(x_i|y)
\end{equation}

The classifications we obtain will obviously depend on the values $P(y)$ and $P(x_i|y)$.  Following Michael et al., we assume $P(y)$ is uniform, and $P(x_i|y)$ is inferred by training on the training set.  Then, once the classifier has been trained, we use it to classify the remaining languages in the test set, and we compare our results to those reported by Michael et al.

### Prepare training and test sets

In [ ]:
# split data into training set vs. test set.
# data in training set has a class label; test set does not.
# create a boolean vector is_training to separate training from test.
# YOUR CODE GOES HERE.
display(is_training)
display(train.head())
display(test.head())
print("Number of languages in training set:", len(train))
print("Number of languages in test set:", len(test))

In [ ]:
# now get training and test data in appropriate format to feed to the model.
# specifically, pull out input feature vectors X, for both training and testing, separately.
# and similarly pull out output classes y, for both training and testing, separately.
X_train, X_test = train.loc[:,phoneme_columns],  test.loc[:,phoneme_columns]
y_train,  y_test = train.loc[:,"label"],  test.loc[:,"label"]
display(X_train.head())
display(y_train.head())

### Train the model.

There are [several variants](https://scikit-learn.org/stable/modules/naive_bayes.html) of naive Bayes available in the scikit-learn library.  We will be using the <span style="font-family:Courier New">BernoulliNB</span> routine, which is [Bernoulli naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB), following Michael et al., p. 70.  We are asking that the prior over classes be uniform, not fit to the data, following Michael et al., p. 70.  An important difference is that we will use Laplace (add-one) smoothing, rather than regularizing with a beta distribution as Michael et al. did, because the routine we use does not (at least not obviously) support the exact method used by Michael et al.

In [ ]:
# train the model.
model = BernoulliNB(fit_prior=False, class_prior=[0.5,0.5])
fit_model = model.fit(X_train, y_train)

### Now examine the results.

Now that the model has been trained, let's first determine which features (phonemes) are distinctively associated with, and against, the Andean core.  This is for comparison with Table 1 (p. 20 of PDF) of Michael et al. 

We require a good measure of distinctive association.  We will use the notion of **representativeness**, as formalized by [Tenenbaum and Griffiths 2001](https://web.mit.edu/cocosci/Papers/cogsci01_final.pdf).  They took the representativeness $R(d,h)$ of data $d$ for hypothesis $h$ to be the evidence that $d$ provides for $h$, relative to the alternative(s) to $h$.  They formalized this idea as the log of the likelihood ratio:

\begin{equation}
R(d,h) = \log \frac{p(d|h)}{p(d| \neg h)} = \log p(d|h) - \log p(d| \neg h)
\end{equation}

$R(d,h)$ is positive if $d$ is distinctively associated with $h$, negative if $d$ is distinctively associated with $\neg h$, and 0 if $d$ is neutral with respect to $h$ and $\neg h$.

We will ask how representative different features (phonemes) are of the Andean core, or class $A$.  This means that for each feature $f$, we wish to find:

\begin{equation}
R(f,A) = \log p(f|A) - \log p(f| \neg A)
\end{equation}

Happily for us, the relevant quantities $\log p(f|A)$ and $\log p(f| \neg A)$ are provided by <span style="font-family:Courier">feature_log_prob_</span> within <span style="font-family:Courier">BernoulliNB</span>.

In [ ]:
# feature_log_prob_ yields empirical log probability of features given a class, i.e. log P(x_i|y).
# see documentation at:
# https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html
# this is a list (len 2) of lists (len 304): outer list corresponds to class y, and inner list holds
# log P(x_i|y) for each feature (phoneme) x_i.
# now use this to create a dict r that holds, for each phoneme, its representativeness of the Andean core, 
# as defined above.

# YOUR CODE GOES HERE.
r

In [ ]:
# the following "andean" and "non-andean" phonemes are taken from michael et al, table 1.
# they are not the full contents of that table, but only a part.  feel free to supplement!
some_andean_phonemes = ['pʰ', 'pʼ', 'ʈʰ', 'ʈʼ', 'kʰ', 'kʼ', 'q', 'qʰ', 'qʼ', 'tʃ', 'tʃʰ', 'tʃʼ']
some_non_andean_phonemes = ['d', 'kʷ', 'ʔ', 'dʒ', 'h', 'ŋʷ', 'ĩ', 'ĩː']

# now put these phonemes together in a df, along with their r values.
# "andean" vs "non-andean" is a classification made by michael et al., where as the r values denote
# how andean-core-like our classifier takes these phonemes to be.

# YOUR CODE GOES HERE.
df

In [ ]:
# now plot how representative these phonemes are of the Andean core, according to the classifier,
# as a function of which category they fell in according to Michael et al.
ax = sns.stripplot(x="R", y="Class", data=df)
ax.axvline(x=0, color='green')
plt.show()

### Replication achieved w.r.t. phoneme status!

That is, our classifer agrees with Michael et al. w.r.t. which phonemes are distinctively associated with the Andean core, and which are distinctively associated with the non-core &mdash; at least for the specific phonemes we examined above.  So far, so good.

Now what about the languages in the test set?  How are they classified by our classifier, based on their phoneme inventories?  In other words, returning to the "core plus periphery" idea, which languages in the periphery (our test set) are "core-like" in their phoneme inventories?

This is easy to find out.  For each language in the test set, with feature vector $X$, we ask how "core-like" our classifier considers it to be by computing $P(y=A | X)$.  Luckily for us, <span style="font-family:Courier">predict_proba(X)</span> within <span style="font-family:Courier">BernoulliNB</span> returns probability estimates for the test vector X, i.e. exactly the quantity we want.

In [ ]:
# for each language in the test set, find its probability of being classified as "core" vs. not.
# use predict_proba(X) - see documentation:
# https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html
# then create df y_pred_probs containing, for each item X in test set, p(A|X) and p(C|X).

# YOUR CODE GOES HERE.

y_pred_probs

In [ ]:
# now merge these predicted class probabilities into the df test that we created earlier.
# first remind yourself what test looks like.
test

In [ ]:
# now merge in the results of running our classifier on the test set.
results = test.reset_index(drop=True).merge(y_pred_probs, left_index=True, right_index=True)
results

### Now consider each language as a whole

We now want to compare the NBC score for each language (as reported by Michael et al. in their Appendix C) with our classifier's output (as obtained here), and to do so across all languages in the test set.  There is a missing link here: the Michael et al. NBC score is their variable $s$ as defined in Equation 1 on their p. 12, whereas our output is $P(y|X)$.  Fortunately these two quantities are easily linked.  Michael et al. point out (pp. 14-15) that in the 2-class case, which we are in:

\begin{equation}
P(y=A|X) = \frac{1}{1+\exp(-s)}
\end{equation}

So we will now apply that transformation to get values for $P(y|X)$ that are based on Michael et al.'s reported NBC (or $s$) scores, and compare those to our values for $P(y|X)$, across languages.

In [ ]:
# apply the above transformation, store results in df results.

# YOUR CODE GOES HERE.

# rt = results, trimmed
rt = results[['name', 'iso', 'label', 'NBC_score', 'probA', 'A']]
rt

In [ ]:
# spot-check a few lgs that michael et al. consider strongly vs. weakly "core-like", from p. 22.
# some of these, e.g. 'ona', are actuallly not in the test set but rather training set.
# we have results here only for test set items.
strong = ['cbi', 'ccc', 'ame', 'vil', 'mca', 'cag', 'ona', 'ona_mtr', 'pue', 'teh']
weak = ['kbh', 'jeb', 'cbu', 'cpc', 'cpu', 'mtp', 'tob_tks', 'tob_lng', 'moc', 'alc_nth']
print("Languages considered by Michael et al. to be strongly core-like:")
display(rt[rt.iso.isin(strong)])
print("Languages considered by Michael et al. to be weakly core-like:")
display(rt[rt.iso.isin(weak)])

### Consider all languages, and compare our results to those of Michael et al.

In the spot-check above, we find that our results match those of Michael et al. well for some languages, but not so well for others.  This should not be entirely surprising, since the specifics our treatment necessarily deviated from theirs, as noted above.  

Given that there both similarities and differences, it would be good to get an overall sense of how well or poorly the two sets of results align.

In [ ]:
# how well do the results of michael et al. (probA) correlate with ours (A)?
rt[["probA", "A"]].corr(method="pearson")

### Observations and conclusions

Please specify what you found (your observations), and separately, what broader inferences you may draw from that (your conclusions).